## Difficult estimation

In [177]:
import numpy as np
import random
from pysat.solvers import Solver
from collections import deque
from scipy.ndimage import label
from scipy.spatial import distance
import matplotlib.pyplot as plt
import itertools 
import z3
import json

In [178]:
dataset = np.load("star_battle_dataset.npz")
puzzles = dataset["puzzles"]
solutions = dataset["solutions"]

In [179]:
# Show 5 puzzles
for i in range(len(puzzles)):
    print(f"Showing puzzle {i+1}/{len(puzzles)}")
    print(puzzles[i])
    print(solutions[i])

Showing puzzle 1/20
[[9 1 1 1 1 7 7 7 7 7]
 [9 9 1 1 1 7 7 7 7 7]
 [9 9 1 1 1 1 6 7 0 0]
 [9 1 1 1 1 1 6 0 0 0]
 [9 2 1 1 1 6 6 0 0 0]
 [2 2 8 8 3 3 6 0 0 0]
 [2 2 8 8 3 3 3 4 0 0]
 [2 2 8 4 4 4 4 4 4 4]
 [2 2 8 5 5 5 5 4 4 4]
 [2 2 5 5 5 5 5 4 4 4]]
[[0 0 0 0 0 0 0 1 0 1]
 [0 1 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 1 0]
 [1 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 1 0]
 [1 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 1 0 0 0]
 [0 0 1 0 0 0 0 0 0 1]
 [0 0 0 0 0 1 0 1 0 0]
 [0 1 0 1 0 0 0 0 0 0]]
Showing puzzle 2/20
[[1 1 1 1 6 6 6 6 6 6]
 [1 1 1 7 6 6 6 6 6 5]
 [1 1 1 7 0 0 0 0 5 5]
 [1 1 1 7 3 0 4 5 5 5]
 [1 1 1 3 3 3 4 5 5 5]
 [1 3 3 3 3 3 4 4 5 5]
 [3 3 3 3 2 2 4 4 4 4]
 [3 3 3 3 2 2 4 4 9 9]
 [8 8 8 3 2 2 2 9 9 9]
 [8 8 8 8 2 2 2 2 9 9]]
[[0 0 0 0 0 1 0 0 1 0]
 [0 1 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 1 0 0]
 [0 0 0 1 0 0 0 0 0 1]
 [1 0 0 0 0 0 1 0 0 0]
 [0 0 1 0 0 0 0 0 1 0]
 [0 0 0 0 1 0 1 0 0 0]
 [0 1 0 0 0 0 0 0 0 1]
 [0 0 0 0 1 0 0 1 0 0]
 [1 0 1 0 0 0 0 0 0 0]]
Showing puzzle 3/20
[[0 0 1 1 1 1 1 

In [221]:
def classify_difficulty(puzzle, solution):
    #Classifies a Star Battle puzzle into Normal, Hard, or Extreme
    answer = np.empty_like(puzzles[0], dtype=object)
    answer[:] = None    
    potential_stars = np.zeros_like(puzzle)
    num_regions = 10


    if normal_solver(puzzle, solution, answer):
        return "Normal"
    # elif hard_solver(puzzle, solution, answer):
    #     return "Hard"
    else:
        return "Extreme"

def normal_solver(puzzle, solution, answer):
    #attempts to solve using normal heuristics
    for _ in range(20):  # Limit iterations to simulate human solving
        made_progress = False

        # Call heuristic functions and capture both progress and updated answer
        made_progress, answer = apply_straight_lines(puzzle, solution, answer)
        made_progress, answer = apply_t_shapes(puzzle, solution, answer)
        made_progress, answer = apply_l_shapes(puzzle, solution, answer)
        made_progress, answer = apply_pbdq_shapes(puzzle, solution, answer)
        made_progress, answer = apply_offset_stack(puzzle, solution, answer)
        made_progress, answer = apply_simple_center_eliminations(puzzle, solution, answer)

        # If the answer is solved, return early
        if np.array_equal(answer, solution):
            return True

        # No progress made, break out and try the hard solver
        if not made_progress:
            break  # No more easy deductions, try hard

    answer_copy = np.copy(answer)  
    answer_copy[answer_copy != 1] = 0  # Set any value not equal to 1 to 0
    
    return np.array_equal(answer_copy, solution)  # Check if puzzle is fully solved

def hard_solver(puzzle, solution, answer):
    #attempts solve using normal and hard heuristics
    for _ in range(10):  # limited iterations
        made_progress = False

        made_progress |= apply_rule_of_crowding(puzzle, solution, answer)
        made_progress |= apply_rule_of_four_squares(puzzle, solution, answer)
        made_progress |= apply_rule_of_clumps(puzzle, solution, answer)
        made_progress |= normal_solver(puzzle,solution, answer)

        if not made_progress:
            break  
        
        
    answer_copy = np.copy(answer)  
    answer_copy[answer_copy != 1] = 0  # Set any value not equal to 1 to 0
    
    return np.array_equal(answer_copy, solution)  # Check if puzzle is fully solved

def zero_surrounding_cells(answer):
    # Get the dimensions of the answer grid
    rows, cols = answer.shape
    
    # Iterate through each cell in the grid
    for row in range(rows):
        for col in range(cols):
            # If there's a star (1) at this position
            if answer[row, col] == 1:
                # Set the surrounding cells to 0 (orthogonally and diagonally)
                for d_row in [-1, 0, 1]:
                    for d_col in [-1, 0, 1]:
                        # Skip the star itself (the current cell)
                        if d_row == 0 and d_col == 0:
                            continue
                        
                        # Calculate the neighboring cell's coordinates
                        new_row, new_col = row + d_row, col + d_col
                        
                        # Check if the neighbor is within bounds
                        if 0 <= new_row < rows and 0 <= new_col < cols:
                            answer[new_row, new_col] = 0

    return answer


#function to update the puzzle with the stars with no cell "removed"]
def mark_no_star_cells(puzzle, answer):
    # Find all cells that are definitely no stars
    no_star_cells = np.argwhere(answer == 0)

    # Create a mask where no-star cells are marked as True
    no_star_mask = np.zeros_like(answer, dtype=bool)
    for cell in no_star_cells:
        row, col = cell
        no_star_mask[row, col] = True
    
    return no_star_mask



# specific heuristics functions 

def apply_straight_lines(puzzle, answer):
    no_star_mask = mark_no_star_cells(puzzle, answer)
    
    # straight line and 3 cell straight line
    made_progress = False
    rows, cols = puzzle.shape

    # Iterate over each region number
    for region in np.unique(puzzle):
        # Find the cells belonging to this region
        region_cells = np.argwhere(puzzle == region)
        
        # Skip processing if any cell in the region is a no-star cell
        region_cells = [cell for cell in region_cells if not no_star_mask[cell[0], cell[1]]]

        # Check if it's a straight line (same row or same column)
        row_coords, col_coords = zip(*region_cells)
        is_row = len(set(row_coords)) == 1
        is_col = len(set(col_coords)) == 1

        if is_row or is_col:
            if is_row:
                row = row_coords[0]
                sorted_cols = sorted(col_coords)

                # Mark all non-region cells in this row as definite zeros
                for c in range(cols):
                    if puzzle[row, c] != region:
                        answer[row, c] = 0
                        made_progress = True

                # If it's a 3-cell straight line, place stars at both ends
                if len(region_cells) == 3:
                    col_min, col_mid, col_max = sorted_cols
                    answer[row, col_min] = 1
                    answer[row, col_max] = 1
                    answer[row, col_mid] = 0  # Middle cell is a definite zero
                    made_progress = True

            elif is_col:
                col = col_coords[0]
                sorted_rows = sorted(row_coords)

                # Mark all non-region cells in this column as definite zeros
                for r in range(rows):
                    if puzzle[r, col] != region:
                        answer[r, col] = 0
                        made_progress = True

                # If it's a 3-cell straight line, place stars at both ends
                if len(region_cells) == 3:
                    row_min, row_mid, row_max = sorted_rows
                    answer[row_min, col] = 1
                    answer[row_max, col] = 1
                    answer[row_mid, col] = 0  # Middle cell is a definite zero
                    made_progress = True

    zero_surrounding_cells(answer)
    return made_progress, answer
    


def apply_t_shapes(puzzle, answer):
    no_star_mask = mark_no_star_cells(puzzle, answer)    

    made_progress = False
    rows, cols = puzzle.shape

    # Iterate over each unique region
    for region in np.unique(puzzle):
        # Get all cells belonging to this region
        region_cells = np.argwhere(puzzle == region)
        
        # Skip processing if any cell in the region is a no-star cell
        region_cells = [cell for cell in region_cells if not no_star_mask[cell[0], cell[1]]]

        # A T-shape must have exactly 4 cells
        if len(region_cells) != 4:
            continue  

        # Convert to a set for easy lookup
        region_set = {tuple(cell) for cell in region_cells}

        # Check all possible T-shape formations
        for r, c in region_cells:
            # Horizontal (Three in a row, one below)
            if {(r, c - 1), (r, c), (r, c + 1), (r + 1, c)}.issubset(region_set):
                # Place stars at both ends of the row, zero the center & bottom
                answer[r, c - 1] = 1
                answer[r, c + 1] = 1
                answer[r, c] = 0
                answer[r + 1, c] = 0
                made_progress = True

            # Horizontal (Three in a row, one above)
            if {(r - 1, c), (r, c - 1), (r, c), (r, c + 1)}.issubset(region_set):
                answer[r, c - 1] = 1
                answer[r, c + 1] = 1
                answer[r, c] = 0
                answer[r - 1, c] = 0
                made_progress = True

            # Vertical (Three in a column, one to the right)
            if {(r - 1, c), (r, c), (r + 1, c), (r, c + 1)}.issubset(region_set):
                answer[r - 1, c] = 1
                answer[r + 1, c] = 1
                answer[r, c] = 0
                answer[r, c + 1] = 0
                made_progress = True

            # Vertical (Three in a column, one to the left)
            if {(r - 1, c), (r, c), (r + 1, c), (r, c - 1)}.issubset(region_set):
                answer[r - 1, c] = 1
                answer[r + 1, c] = 1
                answer[r, c] = 0
                answer[r, c - 1] = 0
                made_progress = True
    
    zero_surrounding_cells(answer)
    return made_progress, answer


# normal heuristic functions
def apply_l_shapes(puzzle, answer):
    no_star_mask = mark_no_star_cells(puzzle, answer)

    made_progress = False

    rows, cols = puzzle.shape

    # Iterate over each unique region
    for region in np.unique(puzzle):
        # all cells in the region
        region_cells = np.argwhere(puzzle == region)
        
        # Skip processing if any cell in the region is a no-star cell
        region_cells = [cell for cell in region_cells if not no_star_mask[cell[0], cell[1]]]

        # An L-shape must have exactly 4 cells
        if len(region_cells) != 4:
            continue  

        # Convert to a set for easy lookup
        region_set = {tuple(cell) for cell in region_cells}

        for r, c in region_cells:

            # Vertical line + right extension
            if {(r - 1, c), (r, c), (r + 1, c), (r + 1, c + 1)}.issubset(region_set):
                answer[r - 1, c] = 1
                answer[r + 1, c] = 1
                answer[r + 1, c + 1] = 0
                answer[r, c] = 0
                made_progress = True

            # Vertical line + left extension
            if {(r - 1, c), (r, c), (r + 1, c), (r + 1, c - 1)}.issubset(region_set):
                answer[r - 1, c] = 1
                answer[r + 1, c] = 1
                answer[r + 1, c - 1] = 0
                answer[r, c] = 0
                made_progress = True

            # Horizontal line + bottom extension
            if {(r, c - 1), (r, c), (r, c + 1), (r + 1, c - 1)}.issubset(region_set):
                answer[r, c - 1] = 1
                answer[r, c + 1] = 1
                answer[r + 1, c - 1] = 0
                answer[r, c] = 0
                made_progress = True

            # Horizontal line + bottom extension
            if {(r, c - 1), (r, c), (r, c + 1), (r + 1, c + 1)}.issubset(region_set):
                answer[r, c - 1] = 1
                answer[r, c + 1] = 1
                answer[r + 1, c + 1] = 0
                answer[r, c] = 0
                made_progress = True

    zero_surrounding_cells(answer)
    return made_progress, answer


def apply_pbdq_shapes(puzzle, answer):
    no_star_mask =  mark_no_star_cells(puzzle, answer)

    made_progress = False

    rows, cols = puzzle.shape

    # Iterate over each unique region
    for region in np.unique(puzzle):
        # all cells in the region
        region_cells = np.argwhere(puzzle == region)
        
        # Skip processing if any cell in the region is a no-star cell
        region_cells = [cell for cell in region_cells if not no_star_mask[cell[0], cell[1]]]

        #  shape must have exactly 5 cells
        if len(region_cells) != 5:
            continue

        # Convert to a set for easy lookup
        region_set = {tuple(cell) for cell in region_cells}

        for r, c in region_cells:
            # Check for a 2x2 square with a top-left extension
            if {(r, c), (r + 1, c), (r, c + 1), (r + 1, c + 1)}.issubset(region_set):
                # Check the 5th cell in the orthogonal direction (top, bottom, left, right)
                if (r - 1, c) in region_set:  # Top extension
                    answer[r - 1, c] = 1
                    made_progress = True
                elif (r + 2, c) in region_set:  # Bottom extension
                    answer[r + 2, c] = 1
                    made_progress = True
                elif (r, c - 1) in region_set:  # Left extension
                    answer[r, c - 1] = 1
                    made_progress = True
                elif (r, c + 2) in region_set:  # Right extension
                    answer[r, c + 2] = 1
                    made_progress = True

            # Check for a 2x2 square with a top-right extension
            if {(r, c), (r + 1, c), (r, c + 1), (r + 1, c + 1)}.issubset(region_set):
                if (r - 1, c + 1) in region_set:  # Top-right extension
                    answer[r - 1, c + 1] = 1
                    made_progress = True
                elif (r + 2, c + 1) in region_set:  # Bottom-right extension
                    answer[r + 2, c + 1] = 1
                    made_progress = True
                elif (r + 1, c - 1) in region_set:  # Top-left extension
                    answer[r + 1, c - 1] = 1
                    made_progress = True
                elif (r + 1, c + 2) in region_set:  # Bottom-left extension
                    answer[r + 1, c + 2] = 1
                    made_progress = True

    zero_surrounding_cells(answer)
    return made_progress, answer


def apply_offset_stack(puzzle, answer):
    no_star_mask = mark_no_star_cells(puzzle, answer)
    
    made_progress = False
    
    # Iterate over each unique region
    for region in np.unique(puzzle):
        # Get all cells in the region
        region_cells = np.argwhere(puzzle == region)
        region_cells = [cell for cell in region_cells if not no_star_mask[cell[0], cell[1]]]

        

        # The shape must have exactly 4 cells
        if len(region_cells) != 4:
            continue

        # Convert to a set for easy lookup
        region_set = {tuple(cell) for cell in region_cells}

        for r, c in region_cells:
            # Check for a Z-shape (horizontal)
            if {(r, c), (r, c + 1), (r + 1, c - 1), (r + 1, c)}.issubset(region_set):
                answer[r, c] = 1
                answer[r + 1, c - 1] = 1
                made_progress = True

            # Check for an S-shape (horizontal)
            if {(r, c), (r, c + 1), (r + 1, c + 1), (r + 1, c + 2)}.issubset(region_set):
                answer[r, c] = 1
                answer[r + 1, c + 2] = 1
                made_progress = True

            # Check for a Z-shape (vertical)
            if {(r, c), (r + 1, c), (r + 1, c + 1), (r + 2, c + 1)}.issubset(region_set):
                answer[r, c] = 1
                answer[r + 2, c + 1] = 1
                made_progress = True

            # Check for an S-shape (vertical)
            if {(r, c), (r + 1, c), (r + 1, c - 1), (r + 2, c - 1)}.issubset(region_set):
                answer[r, c] = 1
                answer[r + 2, c - 1] = 1
                made_progress = True
    zero_surrounding_cells(answer)
    return made_progress, answer


def apply_simple_center_eliminations(puzzle, answer):
    made_progress = False
    no_star_mask = mark_no_star_cells(puzzle, answer)

    # Ensure answer is a NumPy array
    answer = np.array(answer)

    for region in np.unique(puzzle):
        region_cells = np.argwhere(puzzle == region)

        # Apply no-star filtering and convert back to NumPy array
        region_cells = np.array([cell for cell in region_cells if not no_star_mask[cell[0], cell[1]]])

        # Skip empty regions
        if region_cells.size == 0:
            continue

        # Get min/max row and col bounds
        min_r, max_r = region_cells[:, 0].min(), region_cells[:, 0].max()
        min_c, max_c = region_cells[:, 1].min(), region_cells[:, 1].max()

        height = max_r - min_r + 1
        width = max_c - min_c + 1

        center_cells = []

        # Check for a valid 6-cell rectangle (3x2 or 2x3)
        if len(region_cells) == 6 and {height, width} == {3, 2}:
            center_cells = [(min_r + 1, min_c), (min_r + 1, min_c + 1)]  # Vertical middle row

        elif len(region_cells) == 6 and {height, width} == {2, 3}:
            center_cells = [(min_r, min_c + 1), (min_r + 1, min_c + 1)]  # Horizontal middle column

        # Check for a valid 9-cell square (3x3)
        elif len(region_cells) == 9 and height == 3 and width == 3:
            center_cells = [(min_r + 1, min_c + 1)]  # Single center cell

        else:
            continue  # Skip if the region does not match the expected patterns

        # Mark center cells as no star
        for r, c in center_cells:
            if answer[r, c] != 0:  # Only modify if not already marked
                answer[r, c] = 0
                made_progress = True

    zero_surrounding_cells(answer)
    return made_progress, answer





def apply_star_surrounds(puzzle, answer):
    # clears cells surrounding stars, and check if row, column or region is full of stars.
    return False 


# hard heuristic functions

def apply_rule_of_crowding(puzzle, answer):
    # Implement Rule of Crowding logic
    return False  

def apply_rule_of_four_squares(puzzle, answer):
    # Implement Rule of Four-Squares logic
    return False  

def apply_rule_of_clumps(puzzle, answer):
    # Implement Rule of Clumps logic
    return False  



# testing for the functions


In [181]:
#layout function to quickly create a grid
def layout(grid_size=10, num_regions=10):
# random region layout with minimum 3 cells in a region
    grid = np.full((grid_size, grid_size), -1) 

    # place initial 3 cell regions
    region_seeds = [] 
    used_positions = set()  # to avoid overlapping regions

    for i in range(num_regions):
        while True:
            # Pick a random starting cell
            r, c = random.randint(0, grid_size - 1), random.randint(0, grid_size - 1)
            direction = random.choice([(0, 1), (1, 0)])  # (0,1) = horizontal, (1,0) = vertical

            # compute 3-cell region
            cells = [(r + direction[0] * j, c + direction[1] * j) for j in range(3)]

            # checks all cells are within bounds and not overlapping
            if all(0 <= nr < grid_size and 0 <= nc < grid_size and (nr, nc) not in used_positions for nr, nc in cells):
                for nr, nc in cells:
                    grid[nr, nc] = i  # assigns region ID
                    used_positions.add((nr, nc))
                    region_seeds.append((nr, nc))  # adds each part of the seed for later expansion
                break  # exits loop when valid seed region is found

    # expands all regions with flood fill
    def get_neighbors(r, c):
        neighbors = []
        for dr, dc in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            nr, nc = r + dr, c + dc
            if 0 <= nr < grid_size and 0 <= nc < grid_size and grid[nr, nc] == -1:
                neighbors.append((nr, nc))
        return neighbors

    cells_to_expand = region_seeds[:]  # start expanding from the initial 3-cell regions
    while cells_to_expand:
        r, c = cells_to_expand.pop(random.randint(0, len(cells_to_expand) - 1))
        neighbors = get_neighbors(r, c)
        if neighbors:
            random.shuffle(neighbors)
            for nr, nc in neighbors:
                grid[nr, nc] = grid[r, c]  # expand current region
                cells_to_expand.append((nr, nc))  # continue expansion

    return grid

In [182]:
answer = np.empty_like(puzzles[0], dtype=object)  # Use dtype=object for None or custom placeholders
answer[:] = None
answer_copy = np.copy(answer)  # Create a copy of the answer array
answer_copy[answer_copy != 1] = 0  # Set any value not equal to 1 to 0


print(answer)
print(answer_copy)

[[None None None None None None None None None None]
 [None None None None None None None None None None]
 [None None None None None None None None None None]
 [None None None None None None None None None None]
 [None None None None None None None None None None]
 [None None None None None None None None None None]
 [None None None None None None None None None None]
 [None None None None None None None None None None]
 [None None None None None None None None None None]
 [None None None None None None None None None None]]
[[0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]]


In [183]:
answer2 = np.array([
    [-1, -1, -1, -1, -1],
    [-1, -1, 1, -1, -1],
    [-1, -1, -1, -1, -1],
    [-1, -1, -1, -1, -1],
    [-1, -1, -1, -1, -1]
])

print(answer2)

answer_copy2 = zero_surrounding_cells(answer2.copy())  # Call the function on a copy
print(answer_copy2)


[[-1 -1 -1 -1 -1]
 [-1 -1  1 -1 -1]
 [-1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1]]
[[-1  0  0  0 -1]
 [-1  0  1  0 -1]
 [-1  0  0  0 -1]
 [-1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1]]


In [202]:
puzzletry = puzzles[1].copy()
print(puzzletry)
solutiontry = solutions[1].copy()
print(solutiontry)
egpuzzle = mark_no_star_cells(puzzletry, solutiontry)
print(egpuzzle)

[[1 1 1 1 6 6 6 6 6 6]
 [1 1 1 7 6 6 6 6 6 5]
 [1 1 1 7 0 0 0 0 5 5]
 [1 1 1 7 3 0 4 5 5 5]
 [1 1 1 3 3 3 4 5 5 5]
 [1 3 3 3 3 3 4 4 5 5]
 [3 3 3 3 2 2 4 4 4 4]
 [3 3 3 3 2 2 4 4 9 9]
 [8 8 8 3 2 2 2 9 9 9]
 [8 8 8 8 2 2 2 2 9 9]]
[[0 0 0 0 0 1 0 0 1 0]
 [0 1 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 1 0 0]
 [0 0 0 1 0 0 0 0 0 1]
 [1 0 0 0 0 0 1 0 0 0]
 [0 0 1 0 0 0 0 0 1 0]
 [0 0 0 0 1 0 1 0 0 0]
 [0 1 0 0 0 0 0 0 0 1]
 [0 0 0 0 1 0 0 1 0 0]
 [1 0 1 0 0 0 0 0 0 0]]
[[ True  True  True  True  True False  True  True False  True]
 [ True False  True False  True  True  True  True  True  True]
 [ True  True  True  True  True False  True False  True  True]
 [ True  True  True False  True  True  True  True  True False]
 [False  True  True  True  True  True False  True  True  True]
 [ True  True False  True  True  True  True  True False  True]
 [ True  True  True  True False  True False  True  True  True]
 [ True False  True  True  True  True  True  True  True False]
 [ True  True  True  True False  T

In [185]:
for i in range(len(puzzles)): 

    puzzle = puzzles[i]
    solution = solutions[i]

    # Reset tracking arrays for each puzzle
    answer = np.empty_like(puzzles[0], dtype=object)  # Use dtype=object for None or custom placeholders
    answer[:] = None    
    print(apply_straight_lines(puzzles[i], answer)[0])
    
print(puzzles[0])
print(puzzles[1])

False
True
True
False
True
False
True
True
False
True
True
False
True
False
True
True
False
True
False
False
[[9 1 1 1 1 7 7 7 7 7]
 [9 9 1 1 1 7 7 7 7 7]
 [9 9 1 1 1 1 6 7 0 0]
 [9 1 1 1 1 1 6 0 0 0]
 [9 2 1 1 1 6 6 0 0 0]
 [2 2 8 8 3 3 6 0 0 0]
 [2 2 8 8 3 3 3 4 0 0]
 [2 2 8 4 4 4 4 4 4 4]
 [2 2 8 5 5 5 5 4 4 4]
 [2 2 5 5 5 5 5 4 4 4]]
[[1 1 1 1 6 6 6 6 6 6]
 [1 1 1 7 6 6 6 6 6 5]
 [1 1 1 7 0 0 0 0 5 5]
 [1 1 1 7 3 0 4 5 5 5]
 [1 1 1 3 3 3 4 5 5 5]
 [1 3 3 3 3 3 4 4 5 5]
 [3 3 3 3 2 2 4 4 4 4]
 [3 3 3 3 2 2 4 4 9 9]
 [8 8 8 3 2 2 2 9 9 9]
 [8 8 8 8 2 2 2 2 9 9]]


In [186]:
for i in range(len(puzzles)): 

    puzzle = puzzles[i]
    solution = solutions[i]

    # Reset tracking arrays for each puzzle
    answer = np.empty_like(puzzles[0], dtype=object)  # Use dtype=object for None or custom placeholders
    answer[:] = None
    
    
    print(apply_l_shapes(puzzles[i], answer)[0])
    
print(puzzles[10])
print(puzzles[11])



False
False
False
True
False
False
False
False
True
False
False
False
False
False
False
False
False
True
False
True
[[3 3 3 2 4 4 4 4 4 4]
 [3 3 2 2 2 4 4 4 4 5]
 [6 3 2 2 2 4 0 5 5 5]
 [6 6 2 2 2 4 0 5 5 5]
 [6 6 6 2 4 4 0 1 1 5]
 [6 6 4 4 4 4 1 1 1 9]
 [6 8 4 4 4 4 1 1 9 9]
 [8 8 8 7 7 7 1 1 9 9]
 [8 8 8 8 7 7 1 9 9 9]
 [8 8 8 8 7 7 1 9 9 9]]
[[3 3 2 2 2 2 8 8 8 8]
 [3 3 2 2 2 2 2 8 8 8]
 [3 3 3 7 7 7 7 8 8 8]
 [3 3 4 4 7 7 8 8 8 8]
 [4 4 4 4 4 7 9 8 8 8]
 [4 4 5 4 9 9 9 9 9 9]
 [1 5 5 5 5 9 9 0 0 0]
 [1 1 5 5 5 5 0 0 0 0]
 [1 5 5 5 5 6 6 6 0 0]
 [1 1 5 5 5 5 6 0 0 0]]


In [193]:
for i in range(len(puzzles)): 

    puzzle = puzzles[i]
    solution = solutions[i]

    # Reset tracking arrays for each puzzle
    # Reset tracking arrays for each puzzle
    answer = np.empty_like(puzzles[0], dtype=object)  # Use dtype=object for None or custom placeholders
    answer[:] = None
    print(apply_t_shapes(puzzles[i], answer)[0])
    
print(puzzles[18])
print(puzzles[19])

False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
[[3 3 3 3 8 8 8 8 8 8]
 [3 3 3 3 8 8 8 8 8 8]
 [3 3 3 7 7 8 8 8 8 0]
 [3 3 3 7 7 1 0 0 0 0]
 [3 9 7 7 7 1 0 0 0 0]
 [9 9 9 9 1 1 1 4 4 4]
 [9 9 9 9 1 1 2 2 2 4]
 [5 9 9 6 6 6 2 2 2 4]
 [5 5 6 6 6 6 6 2 2 2]
 [5 5 6 6 6 6 2 2 2 2]]
[[4 4 4 6 2 2 2 2 7 7]
 [4 4 6 6 6 2 2 3 7 7]
 [4 4 6 6 6 6 2 3 3 7]
 [4 6 6 6 6 6 3 3 3 7]
 [1 1 1 6 6 3 3 3 9 8]
 [1 1 1 6 6 3 3 3 9 8]
 [0 1 1 6 5 5 3 9 9 8]
 [0 0 0 0 5 5 5 8 8 8]
 [0 0 0 5 5 5 5 8 8 8]
 [0 0 0 5 5 5 5 5 8 8]]


In [194]:
for i in range(len(puzzles)): 

    puzzle = puzzles[i]
    solution = solutions[i]

    # Reset tracking arrays for each puzzle
# Reset tracking arrays for each puzzle
    answer = np.empty_like(puzzles[0], dtype=object)  # Use dtype=object for None or custom placeholders
    answer[:] = None
    print(apply_pbdq_shapes(puzzles[i], answer)[0])
    
print(puzzles[18])
print(puzzles[19])


True
False
True
False
False
True
False
False
True
False
False
False
False
False
False
False
False
True
True
False
[[3 3 3 3 8 8 8 8 8 8]
 [3 3 3 3 8 8 8 8 8 8]
 [3 3 3 7 7 8 8 8 8 0]
 [3 3 3 7 7 1 0 0 0 0]
 [3 9 7 7 7 1 0 0 0 0]
 [9 9 9 9 1 1 1 4 4 4]
 [9 9 9 9 1 1 2 2 2 4]
 [5 9 9 6 6 6 2 2 2 4]
 [5 5 6 6 6 6 6 2 2 2]
 [5 5 6 6 6 6 2 2 2 2]]
[[4 4 4 6 2 2 2 2 7 7]
 [4 4 6 6 6 2 2 3 7 7]
 [4 4 6 6 6 6 2 3 3 7]
 [4 6 6 6 6 6 3 3 3 7]
 [1 1 1 6 6 3 3 3 9 8]
 [1 1 1 6 6 3 3 3 9 8]
 [0 1 1 6 5 5 3 9 9 8]
 [0 0 0 0 5 5 5 8 8 8]
 [0 0 0 5 5 5 5 8 8 8]
 [0 0 0 5 5 5 5 5 8 8]]


In [207]:
for i in range(len(puzzles)): 

    puzzle = puzzles[i]
    solution = solutions[i]

    # Reset tracking arrays for each puzzle
# Reset tracking arrays for each puzzle
    answer = np.empty_like(puzzles[0], dtype=object)  # Use dtype=object for None or custom placeholders
    answer[:] = None
    print(apply_offset_stack(puzzles[i], answer)[0])

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False


In [208]:
#OFSET STACK FUCNTION CHANGES ITS MIND
puzzle = np.array([
        [4, 4, 4, 6, 2, 2, 2, 2, 7, 7],
        [4, 4, 6, 6, 6, 2, 2, 3, 7, 7],
        [4, 4, 6, 6, 6, 6, 2, 3, 3, 7],
        [4, 6, 6, 6, 6, 6, 3, 3, 3, 7],
        [1, 1, 1, 6, 6, 3, 3, 3, 9, 8],
        [1, 1, 1, 6, 6, 3, 3, 9, 9, 8], 
        [0, 1, 1, 6, 5, 5, 3, 9, 8, 8], 
        [0, 0, 0, 0, 5, 5, 5, 8, 8, 8],
        [0, 0, 0, 5, 5, 5, 5, 8, 8, 8],
        [0, 0, 0, 5, 5, 5, 5, 5, 8, 8]
    ])

# Reset tracking arrays for each puzzle
# Reset tracking arrays for each puzzle
answer = np.empty_like(puzzles[0], dtype=object)  # Use dtype=object for None or custom placeholders
answer[:] = None
print(apply_offset_stack(puzzle, answer)[0])

True


In [220]:
for i in range(len(puzzles)): 

    puzzle = puzzles[i]
    solution = solutions[i]

    # Reset tracking arrays for each puzzle
# Reset tracking arrays for each puzzle
    answer = np.empty_like(puzzles[0], dtype=object)  # Use dtype=object for None or custom placeholders
    answer[:] = None
    print(apply_simple_center_eliminations(puzzles[i], answer)[0])
    
print(puzzles[16])
print(apply_simple_center_eliminations(puzzles[16], answer)[1])
print(puzzles[17])

False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
True
False
False
False
[[7 7 6 2 2 2 8 8 8 8]
 [7 7 6 2 2 2 0 8 8 8]
 [7 7 6 2 2 0 0 8 8 8]
 [6 6 6 6 6 0 0 0 8 8]
 [3 3 3 6 0 0 0 0 8 8]
 [3 3 3 3 0 0 0 0 8 8]
 [4 4 3 3 1 9 9 9 9 9]
 [4 4 1 1 1 1 9 9 9 9]
 [4 4 4 1 1 5 5 5 9 9]
 [4 4 4 1 1 5 5 5 5 5]]
[[0 0 None None None None None None None None]
 [0 0 None None None None None None None None]
 [0 0 None None None None None None None None]
 [None None None None None None None None None None]
 [None None None None None None None None None None]
 [None None None None None None None None None None]
 [None None None None None None None None None None]
 [None None None None None None None None None None]
 [None None None None None None None None None None]
 [None None None None None None None None None None]]
[[9 9 9 9 8 8 8 8 8 2]
 [9 9 9 9 0 1 1 4 4 2]
 [9 7 9 9 0 1 1 4 4 2]
 [7 7 7 0 0 1 4 4 4 2]
 [7 7 7 7 1 1 1 4 4 2]
 [7 7 7 7 1 1 1 4 4 2]


# classify

In [ ]:
# Classify each puzzle
difficulty_labels = [classify_difficulty(puzzles[i], solutions[i]) for i in range(len(puzzles))]

# Save labeled data
np.savez_compressed("star_battle_labeled_dataset.npz", puzzles=puzzles, solutions=solutions, labels=np.array(difficulty_labels))
print("Labeled dataset saved successfully!")

In [ ]:
# dataset = np.load("star_battle_dataset.npz")
# puzzles = dataset["puzzles"]
# solutions = dataset["solutions"]

difficulties = []
for i in range(len(puzzles)):
    difficulty_score = classify_difficulty(puzzles[i], solutions[i])
    
    

# Save difficulty labels
print(difficulties)
# np.savez_compressed("star_battle_dataset_with_difficulty.npz", puzzles=puzzles, solutions=solutions, difficulties=difficulties)
# print("Dataset with difficulty scores saved!")


['Hard', 'Hard', 'Hard', 'Hard', 'Hard']
Dataset with difficulty scores saved!
